## Elo Rating method

This is the Elo method.  The example is taken from "Who's #1" by Langville and Meyer and the results match.  Note, the data includes the entire 2009 NFL season, including all playoff games.  

Reference: https://www.gautamnarula.com/rating/

In [1]:
import math 
  
# Function to calculate the Probability 
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 1000)) 
  
# Function to calculate Elo rating 
# K is a constant. 
# Player A wins over Player B.  
# tie = true if tie, false otherwise
def EloRating(Ra, Rb, K, tie): 
    
    # To calculate the Winning 
    # Probability of Player B 
    Pb = Probability(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = Probability(Rb, Ra) 
  
    # Updating the Elo Ratings 
    if tie:
       Ra = Ra + K * (1/2 - Pa) 
       Rb = Rb + K * (1/2 - Pb) 
    else:        
       Ra = Ra + K * (1 - Pa) 
       Rb = Rb + K * (0 - Pb) 
    
    return Ra, Rb

In [2]:
gameFilename = 'week16wd.csv'
teamFilename = '2019teams.txt'

### Load the team names into an array

In [3]:
import pandas as pd

teamNames = pd.read_csv(teamFilename, header = None)
numTeams = len(teamNames)

### Load the games

In [4]:
# columns of games are:
#	column 0 = days since 1/1/0000
#	column 1 = date in YYYYMMDD format
#	column 2 = team1 index
#	column 3 = team1 homefield (1 = home, -1 = away, 0 = neutral)
#	column 4 = team1 score
#	column 5 = team2 index
#	column 6 = team2 homefield (1 = home, -1 = away, 0 = neutral)
#	column 7 = team2 score
games = pd.read_csv(gameFilename, header = None)
games[0].replace(365243, 737600, inplace=True)
numGames = len(games)

### Create Elo ratings

In [5]:
import numpy as np

print(">>>>>>> I'm not including ties.")

eloRatings = np.zeros(numTeams)

for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
    team2Score = games.loc[i, 7]

    if team1Score > team2Score:
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID],32, False)
    elif team1Score < team2Score:
        team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID],32, False)
    else:  
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID],32, True)
        
    eloRatings[team1ID] = team1Rating
    eloRatings[team2ID] = team2Rating

>>>>>>> I'm not including ties.


### Sort and print the ranking of teams

In [6]:
iSort = np.argsort(-eloRatings)

print('\n\n************** ELO Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
for i in range(numTeams):
    print(f'{i+1:4d}   {eloRatings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return



************** ELO Rating Method **************

Rank   Rating      Team   
   1   163.17249   Baltimore
   2   133.68426   New_England
   3   128.82449   Green_Bay
   4   124.99062   San_Francisco
   5   114.72397   New_Orleans
   6   104.91728   Kansas_City
   7   94.87107   Seattle
   8   77.13413   Houston
   9   70.76816   Minnesota
  10   65.94956   Buffalo
  11   16.15170   Tennessee
  12   15.58044   Philadelphia
  13   12.48589   LA_Rams
  14   12.13486   Pittsburgh
  15   -9.96907   Tampa_Bay
  16   -14.97660   Chicago
  17   -17.62954   Dallas
  18   -18.69013   Las_Vegas
  19   -18.74388   Indianapolis
  20   -31.42007   Atlanta
  21   -34.36865   Denver
  22   -37.85211   NY_Jets
  23   -40.36767   Cleveland
  24   -50.53106   Arizona
  25   -75.62071   LA_Chargers
  26   -77.71964   Jacksonville
  27   -77.79417   Carolina
  28   -99.84325   Miami
  29   -100.78293   NY_Giants
  30   -122.24928   Detroit
  31   -124.73787   Washington
  32   -182.09231   Cincinnati



In [7]:
colleyRank = pd.DataFrame(eloRatings)
player = pd.DataFrame(teamNames)
rrank = colleyRank[0]
ewf = player[1]
df = pd.concat([rrank,ewf],axis=1)
dv = df.sort_values(0,ascending=False)
dc = dv.reset_index()
del dc['index']
dc = dc.reset_index()
dc['index'] = dc['index'] + 1
dc = dc. sort_values(1)
dc = dc.rename(columns={'index':'rank'})
dc = dc.rename(columns={1:'teams'})
del dc[0]
dc = dc[['teams','rank']]


In [8]:
de = teamNames.rename(columns={0:'ID', 1:'Name'})
de['ID'].astype(np.int64)
de.set_index('ID')
mydict = dict(zip(de.Name, de.ID))
dc['teams'].replace(mydict, inplace=True)
dc.head()

,teams,rank
23,1,24
19,2,20
0,3,1
9,4,10
26,5,27


In [9]:
def returnRank(t):
    df = dc.loc[dc['teams'] == t]
    return df.iat[0,1]

In [10]:
gamet = pd.read_csv('gamedd.csv', header = None)
weekg = gamet.loc[gamet[0] == 17]
numgweek = len(weekg)
correct = []
incorrect = []
num = 1

In [11]:
for i in range(numgweek):
    tea1ID = weekg.iloc[i, 3]
    tea2ID = weekg.iloc[i, 6]
    tea1Score = weekg.iloc[i, 5]
    tea2Score = weekg.iloc[i, 8]
    tea1Rank = returnRank(tea1ID)
    tea2Rank = returnRank(tea2ID)
    if tea1Score > tea2Score and tea1Rank < tea2Rank:
        correct.append(num)
    else:
        incorrect.append(num)

In [12]:
len(correct)/numgweek

0.5

## 